In [90]:
import pandas as pd

# Creamos las funciones para FastAPI

####  En las funciones de esta seccion se puede ver la evolucion que han tenido y las opciones que se han probado para obtener los diferentes resultados. En algunos casos se encuentran los resultados finales que se pueden ver en FastAPI.py.

## Abrimos tablas procesadas anteriormente

- merged_steam_user_data.parquet

In [91]:
steam_user_content = 'FastAPI/merged_steam_user_data.parquet'
steam_user_data = pd.read_parquet(steam_user_content)

- merged_steam_rev_data.parquet

In [92]:
steam_rev_content = 'FastAPI/merged_steam_rev_data.parquet'
steam_rev_data = pd.read_parquet(steam_rev_content)

### Verificaciones Iniciales

- steam_user_data

In [93]:
steam_user_data.head(10)

,item_id,user_id,playtime_forever,genres,release_year
0,620,76561198062042547,2268,RPG,2010
1,55230,Str1ke4,307,Action,2000
2,34270,76561198040135674,5,Indie,2017
3,34330,hyphyy,2960,Strategy,2017
4,35420,Shykorus_Wilde_Totora,0,Web Publishing,2016
5,24800,Gunstorm246,0,Design & Illustration,2017
6,12220,76561198105011167,2,Action,2016
7,50,fenetic,0,Simulation,2018
8,32370,Doctorsicknote,1408,Action,2017
9,620,captainmehico,692,RPG,2010


In [94]:
steam_user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4916 entries, 0 to 4915
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_id           4916 non-null   int64 
 1   user_id           4916 non-null   object
 2   playtime_forever  4916 non-null   int64 
 3   genres            4916 non-null   object
 4   release_year      4916 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 192.2+ KB


- steam_rev_data

In [95]:
steam_rev_data.head()

,app_name,item_id,release_year,recommend,sentiment_analysis
0,Sword of the Samurai,327950,1989,True,2
1,Commander Keen,9180,1990,True,2
2,LOOM™,32340,1990,True,2
3,Commander Keen,9180,1990,False,0
4,Crystal Caves,358260,1991,True,2


In [96]:
steam_rev_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6288 entries, 0 to 6287
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   app_name            6288 non-null   object
 1   item_id             6288 non-null   int64 
 2   release_year        6288 non-null   int64 
 3   recommend           6288 non-null   bool  
 4   sentiment_analysis  6288 non-null   int64 
dtypes: bool(1), int64(3), object(1)
memory usage: 202.8+ KB


## UserForGenre

In [97]:
def UserForGenre(genero: str = 'Action') -> dict:
    df_filtered_by_genre = steam_user_data[steam_user_data['genres']==genero]

    user_hours = df_filtered_by_genre.groupby('user_id')['playtime_forever'].sum()
    max_time_played_user = user_hours.idxmax()

    max_user_df = df_filtered_by_genre[df_filtered_by_genre['user_id']== max_time_played_user]
    hours_by_year = max_user_df.groupby('release_year')['playtime_forever'].sum().reset_index()
    hours_years_list = [{'Año': year, 'Horas': hours} for year, hours in zip(hours_by_year['release_year'], hours_by_year['playtime_forever'])]

    result ={
        'Usuario con mas horas jugadas para genero {}'.format(genero): max_time_played_user,
        'Horas jugadas': hours_years_list
        }
    

    return result

resultado=UserForGenre("Casual")
print(resultado)
print(UserForGenre('Action'))
print(UserForGenre('Adventure'))

{'Usuario con mas horas jugadas para genero Casual': '76561198035890730', 'Horas jugadas': [{'Año': 2016, 'Horas': 47060}]}
{'Usuario con mas horas jugadas para genero Action': 'leweeeeeeee', 'Horas jugadas': [{'Año': 2010, 'Horas': 146784}]}
{'Usuario con mas horas jugadas para genero Adventure': 'rabbidscool', 'Horas jugadas': [{'Año': 2014, 'Horas': 14921}]}


## PlayTimeGenre

In [98]:
def PlayTimeGenre(genero: str = 'Action') -> dict:
    df_filtered_by_genre = steam_user_data[steam_user_data['genres']==genero]
    user_hours = df_filtered_by_genre.groupby('release_year')['playtime_forever'].sum()
    max_time_played_user = user_hours.idxmax()

    result ={
        "Año de lanzamiento con más horas jugadas para género {}".format(genero): max_time_played_user
        }
    return result

resultado=PlayTimeGenre("Casual")
print(resultado)
print(PlayTimeGenre('Indie'))
print(PlayTimeGenre('Adventure'))

{'Año de lanzamiento con más horas jugadas para género Casual': 2010}
{'Año de lanzamiento con más horas jugadas para género Indie': 2014}
{'Año de lanzamiento con más horas jugadas para género Adventure': 2016}


## UsersRecommend

In [99]:
def UsersRecommend(year: int = '2020') -> dict:
    user_reviews_recommend = steam_rev_data[steam_rev_data['release_year']==year]
    año_recommend=user_reviews_recommend.groupby('item_id')['recommend'].sum()
    usuario_recomienda = año_recommend.idxmax()


    result ={
        "1º Puesto para año {} es para el item".format(year): usuario_recomienda
        }
    return result

resultado=UsersRecommend(2015)
print(resultado)

{'1º Puesto para año 2015 es para el item': 208650}


In [100]:
def UsersRecommend(year: int = 2017) -> dict:
    user_reviews_recommend = steam_rev_data[steam_rev_data['release_year']==year]
    año_recommend=user_reviews_recommend.groupby('app_name')['recommend'].sum()
    usuario_recomienda = año_recommend.idxmax()


    result ={
        "1º Puesto para año {} es para el item".format(year): usuario_recomienda
        }
    return result

resultado=UsersRecommend(2015)
print(resultado)

{'1º Puesto para año 2015 es para el item': '12 Labours of Hercules'}


In [101]:
def Top3RecommendedGames(year: int = 2017) -> dict:
    user_reviews_recommend = steam_rev_data[steam_rev_data['release_year'] == year]
    top_games = user_reviews_recommend.groupby('app_name')['recommend'].sum().nlargest(3)
    top_games_list = top_games.index.tolist()

    result = {
        f"Los tres juegos más recomendados para el año {year} son": top_games_list
    }
    return result

resultado = Top3RecommendedGames(2015)
print(resultado)


{'Los tres juegos más recomendados para el año 2015 son': ['12 Labours of Hercules', '404Sight', '8BitMMO']}


In [102]:
def Top3RecommendedGames(year: int = 2020) -> dict:
    # Filtrar por el año deseado en user_reviews_dataset
    user_reviews_recommend = steam_rev_data[steam_rev_data['release_year'] == year]
    
    # Obtener los juegos más recomendados
    top_games = user_reviews_recommend.groupby('app_name')['recommend'].sum().nlargest(3)
    top_games_list = top_games.index.tolist()
    
    # Enumerar los juegos más jugados
    top_played_ranking = [{"Puesto {}: ".format(i + 1): game} for i, game in enumerate(top_games_list)]

    return top_played_ranking

resultado = Top3RecommendedGames(2015)
print(resultado)


[{'Puesto 1: ': '12 Labours of Hercules'}, {'Puesto 2: ': '404Sight'}, {'Puesto 3: ': '8BitMMO'}]


In [103]:
def Top3RecommendedGames(year: int = 2020) -> list:
    # Filtrar por el año deseado en user_reviews_dataset
    user_reviews_recommend = steam_rev_data[steam_rev_data['release_year'] == year]
    
    # Obtener los juegos más recomendados
    top_games = user_reviews_recommend.groupby('app_name')['recommend'].sum().nlargest(3)
    top_games_list = top_games.index.tolist()
    
    # Enumerar los juegos más jugados
    top_played_ranking = [{"Puesto {}: ".format(i + 1): game} for i, game in enumerate(top_games_list)]

    return top_played_ranking

resultado = Top3RecommendedGames(2015)

print("Mas recomendados:")
for game_dict in resultado:
    for key, value in game_dict.items():
        print(key, value)

Mas recomendados:
Puesto 1:  12 Labours of Hercules
Puesto 2:  404Sight
Puesto 3:  8BitMMO


## UsersNotRecommend

In [104]:
def Top3LeastRecommendedGames(year: int = 2020) -> dict:
    # Filtrar por el año deseado en user_reviews_dataset
    user_reviews_recommend = steam_rev_data[steam_rev_data['release_year'] == year]
    
    # Obtener los juegos menos recomendados usando nsmallest
    least_recommended = user_reviews_recommend.groupby('item_id')['recommend'].sum().nsmallest(3)
    least_recommended_list = least_recommended.index.tolist()

    # Enumerar los juegos menos recomendados
    least_recommended_ranking = [{"Puesto {}: ".format(i + 1): game} for i, game in enumerate(least_recommended_list)]

    return least_recommended_ranking

resultado = Top3LeastRecommendedGames(2014)
print(resultado)


[{'Puesto 1: ': 222420}, {'Puesto 2: ': 235210}, {'Puesto 3: ': 237970}]


In [105]:
def Top3LeastRecommendedGames(year: int = 2020) -> dict:
    # Filtrar por el año deseado en user_reviews_dataset
    user_reviews_recommend = steam_rev_data[steam_rev_data['release_year'] == year]
    
    # Obtener los juegos menos recomendados usando nsmallest
    least_recommended = user_reviews_recommend.groupby('app_name')['recommend'].sum().nsmallest(3)
    least_recommended_list = least_recommended.index.tolist()

    # Enumerar los juegos menos recomendados
    least_recommended_ranking = [{"Puesto {}: ".format(i + 1): game} for i, game in enumerate(least_recommended_list)]

    return least_recommended_ranking

resultado = Top3LeastRecommendedGames(2014)
print(resultado)

[{'Puesto 1: ': '99 Levels To Hell'}, {'Puesto 2: ': 'A.R.E.S. Extinction Agenda EX'}, {'Puesto 3: ': 'Abyss Odyssey'}]


In [106]:
def Top3LeastRecommendedGames(year: int = 2020) -> dict:
    # Filtrar por el año deseado en user_reviews_dataset
    user_reviews_recommend = steam_rev_data[steam_rev_data['release_year'] == year]
    
    # Obtener los juegos menos recomendados usando nsmallest
    least_recommended = user_reviews_recommend.groupby('app_name')['recommend'].sum().nsmallest(3)
    least_recommended_list = least_recommended.index.tolist()

    # Enumerar los juegos menos recomendados
    least_recommended_ranking = [{"Puesto {}: ".format(i + 1): game} for i, game in enumerate(least_recommended_list)]

    return least_recommended_ranking

resultado = Top3LeastRecommendedGames(2014)
print("Mas recomendados:")
for game_dict in resultado:
    for key, value in game_dict.items():
        print(key, value)

Mas recomendados:
Puesto 1:  99 Levels To Hell
Puesto 2:  A.R.E.S. Extinction Agenda EX
Puesto 3:  Abyss Odyssey


In [107]:
def UsersNotRecommend(year: int = 2017) -> dict:
    user_reviews_recommend = steam_rev_data[steam_rev_data['release_year'] == year]
    least_recommended = user_reviews_recommend.groupby('app_name')['recommend'].sum().nsmallest(3)
    least_recommended_list = least_recommended.index.tolist()

    least_recommended_ranking = [{"Puesto {}: ".format(i + 1): str(game)} for i, game in enumerate(least_recommended_list)]

    return least_recommended_ranking
resultado = UsersNotRecommend(2014)
print(resultado)

[{'Puesto 1: ': '99 Levels To Hell'}, {'Puesto 2: ': 'A.R.E.S. Extinction Agenda EX'}, {'Puesto 3: ': 'Abyss Odyssey'}]


In [108]:
def UsersNotRecommend(year: int = 2017):
    user_reviews_recommend = steam_rev_data[steam_rev_data['release_year'] == year]
    least_recommended = user_reviews_recommend.groupby(
        'app_name')['recommend'].sum().nsmallest(3)
    least_recommended_list = least_recommended.index.tolist()
    least_recommended_ranking = {
        f"Puesto {i + 1}": game for i, game in enumerate(least_recommended_list)}

    return least_recommended_ranking


## sentiment_analysis

In [109]:
def sentiment_analysis(year: int = 2020) -> dict:
    year=int(year)
    reviews_by_year = steam_rev_data[steam_rev_data['release_year'] == year]
    counts_by_sentiment = reviews_by_year['sentiment_analysis'].value_counts().fillna(0).astype(int)
    result = {
        'Negativo': counts_by_sentiment.get(0, 0),
        'Neutro': counts_by_sentiment.get(1, 0),
        'Positivo': counts_by_sentiment.get(2, 0)
    }
    return result

resultado = sentiment_analysis(2017)
print(resultado)

{'Negativo': 34, 'Neutro': 29, 'Positivo': 55}


In [110]:
def sentiment_analysis(year: int = 2017) -> dict:
    reviews_by_year = steam_rev_data[steam_rev_data['release_year'] == year]
    counts_by_sentiment = reviews_by_year['sentiment_analysis'].value_counts(
    ).to_dict()
    result = {
        'Negativo': counts_by_sentiment.get(0, 0),
        'Neutro': counts_by_sentiment.get(1, 0),
        'Positivo': counts_by_sentiment.get(2, 0)
    }

    return result

resultado = sentiment_analysis(2017)
print(resultado)

{'Negativo': 34, 'Neutro': 29, 'Positivo': 55}
